# Scrape Webpages in a Folder

In [21]:
import bs4
from bs4 import BeautifulSoup
import re
import json
import glob
from tqdm import tqdm
import pandas as pd

# Declare Parser Lambda

In [22]:
def parser_benefit(content):
    """
    Parse benefit text
        
    Returns:
        None if not found
    """
    benefit = content.find('h2', string=re.compile('公司福利'))
    
    if benefit is not None:
        benefit = benefit.find_next().get_text(strip=True)
        
        
    return benefit


parsers = {'company': lambda content: content.find('header', attrs={'class': ['header', 'mb-0']}).find('a', attrs={'class': 'cn'}).text.strip(),
          'title': lambda content: content.find('header', attrs={'class': ['header', 'mb-0']}).find('h1').find(text=True).strip(),
          'salary': lambda content: content.find('dd', attrs={'class': 'salary'}).find(text=True).strip(),
          'requirement': lambda content: content.find('h2', string='工作內容').find_parent('section').find('p').get_text(strip=True),
          'requirement_others': lambda content: content.find('dt', string=re.compile('其他條件')).find_next().get_text(strip=True),
          'address': lambda content: content.find('dd', attrs={'class': 'addr'}).find(text=True).strip(),
           'experience': lambda content: content.find('dt', string=re.compile('工作經歷')).find_next().get_text(strip=True),
          'benefit': lambda content: parser_benefit(content)}

# Declare Parser Class

In [23]:
class HtmlParser():
    def __init__(self, parsers):
        self.parsers = parsers
        
        
    def parse(self, filename):
        
        with open(filename) as f:
            content = BeautifulSoup(f, "html.parser")
        
        jobObject = {}
        
        for key, parser in self.parsers.items():
            jobObject[key] = parser(content)
            
        return jobObject    
        
        
    def parse_pages(self, path_folder):
        """
        Args:
            path_folder (str): folder path of html files
            
        Returns:
            list of dict
        """
        jobs = []
        
        for f in tqdm(glob.glob(path_folder + "*.html")):
            jobObject = self.parse(f)
            jobs.append(jobObject)
            
        return jobs

# Declare Save Function

In [24]:
def save_job_objects(job_objects, filename, encoding='utf8'):
    """
    Args:
        job_objects (list of dict): list of jobs
    """
    with open(filename, 'w', encoding=encoding) as f:
        json.dump(job_objects, f, ensure_ascii=False)
    

# Main Function

In [25]:
path_folder='./dataset/'

# parse html pages in a folder
jobs = HtmlParser(parsers).parse_pages(path_folder)


100%|██████████| 996/996 [00:47<00:00, 21.14it/s]


In [26]:
print('Number of jobs={}\n'.format(len(jobs)))
print(jobs[:2])

Number of jobs=996

[{'company': '芯鼎科技股份有限公司', 'title': '系統應用工程師-台北(CD2320)', 'salary': '待遇面議                                                                                    （', 'requirement': '- Driver development and fine tune of customized HW accelerator for OpenCV/OpenCL/OpenVX/CNN.- iCatchOS(RTOS) vision process framework development and maintain (SDK/BSP)- Integration and maintain of in-house design algorithms and performance fine tune.- Discuss/Design & co-work with customers for vision related requests.', 'requirement_others': '- A BS degree in Computer Engineering, Electrical Engineering, or Computer Science- At least 1-3 years of embedded software development experience (Senior – more than 5 years)- Familiar with C/C++ programming language, nice to have experience in Assembly language- Knowledge of Real-Time concepts and have RTOS experience (ThreadX , FreeRTOS, eCos, RTLinux, …)- Good understanding of CPU (e.g. ARM, MIPS ..)  architecture- Image processing & analysis algo

In [27]:
filename_json='./dataset/jobs_104.json'

# save job (list of dictionary)
save_job_objects(jobs, filename_json)

# Inspect JSON file using Pandas

In [28]:
job_df = pd.read_json(filename_json)

In [29]:
len(job_df)

996

In [30]:
job_df.head()

,address,benefit,company,experience,requirement,requirement_others,salary,title
0,台北市內湖區內湖路一段244-1號3F (內湖科技園區),福利健全■ 優渥的薪資、獎金◎ 端午節、中秋節、年終獎金。◎ 定期依工作績效調薪。■ 完善的...,芯鼎科技股份有限公司,不拘,- Driver development and fine tune of customiz...,"- A BS degree in Computer Engineering, Electri...",待遇面議 ...,系統應用工程師-台北(CD2320)
1,台北市大安區復興南路一段,＊獎金福利＊1. 年節禮金2. 生日禮金3. 年終分紅4. 人才介紹獎金＊休假福利＊1. 週...,銓鴻資訊有限公司,1年以上,1. 研究市場數據和交易數據，並進行交易策略研發。2. 開發模型和工具，監控分析交易程序運行...,將先以電子履歷篩選增加效率，請注意以下事項：■ 履歷請詳述相關經驗並請提供 Github/S...,"年薪 1,000,000~2,500,000元",計量交易員 Quantitative Trader (台北大安區)
2,新北市新店區全區 中興路三段 88 號 (依照公司規定分派),《薪酬福利＠HTC》我們極力落實利潤與員工分享的精神，透過多元的薪酬組合與完善的福利方案，和...,宏達電 HTC Corporation_宏達國際電子股份有限公司,不拘,Join the creative thinkers at HTC Healthcare. ...,Requirement:-MS or PhD in computer science or ...,待遇面議 ...,(RD S/W) DeepQ - Computer Vision Research Engi...
3,桃園市中壢區大桃園區,正職/全職人員之福利與訓練制度：【薪酬福利】-每年依市場薪資水準評估薪資調整專案-生日禮金(...,OK超商_來來超商股份有限公司,不拘,OK超商認為，用心經營的企業，首重人才培訓。因此，我們投入了大量的訓練資源，積極培育新一代的...,1.具服務熱忱，科系、經驗不拘，須輪班。2.須自備交通工具及電腦網路設備(透過e-Learn...,"月薪 31,000~33,000元",儲備幹部(大桃園地區)
4,新竹縣竹北市復興三路二段168號10樓之2,【工作福利】彈性上下班。【休假福利】週休二日、國定假日、婚假、產假、陪產假、特休假、彈性休假...,神亞科技股份有限公司,3年以上,影像相關之深度學習演算法研究與開發,1. 有開發image processing相關應用經驗2. 熟悉深度學習相關算法（e.g....,待遇面議 ...,AI 工程師


In [31]:
job_df[-10:]

,address,benefit,company,experience,requirement,requirement_others,salary,title
986,新竹市篤行一路2號 (新竹科學工業園區),<完善的在職訓練>公司全力支持同仁追求新知與學習新技術，協助同仁取得全球最新的技術文獻、產業...,聯詠科技股份有限公司,不拘,"1.\t熟悉Linux系統, 具C/C++/Python程式語言設計經驗2.\t熟悉Mach...",擅長工具：C、C++、Python、TensorFlow、Caffe、Keras,待遇面議 ...,AI語音系統軟體開發工程師 (SoC_LTD-SA)
987,台中市北屯區敦化路一段,尚未提供,澳洲商艾萊門窗股份有限公司台灣分公司,5年以上,Report to Global HQ with leadership position a...,"Job Requirements (include, but are not limited...",待遇面議 ...,Senior Manufacturing engineer/資深機械工程師
988,台北市南港區三重路19-3號一樓,〔薪資福利〕- 獎金(分紅/年終)- 勞保、健保、退休金提撥- 彈性上下班- 優於勞基法之給...,願境網訊股份有限公司_KKBOX,不拘,About KKStreamKKStream is a KKBOX branch estab...,Requirement* You have the ability to condense ...,待遇面議 ...,[KKStream] Data Analyst
989,台南市六甲區菁埔里5號,1.勞、健保2.三節禮金3.免費供午膳4.勞退6%提撥5.免費提供制服,世洲機械廠股份有限公司,1年以上,-專科以上機械相關科系畢業-熟AUTOCAD、EXCEL、WORD-具工作經驗者,未填寫,"月薪 28,000~35,000元",機械繪圖人員
990,新竹縣竹東鎮中興路四段195號53館,這是一個為研究菁英打造的研發環境，我們的訴求是讓研究人員能在此構築人生理想及技術成就，讓工作...,工研院 _財團法人工業技術研究院,2年以上,•Strong knowledge and experience with Machine ...,科系:電機工程、資訊工程等相關科系。工作經驗:1.工作2年以上尤佳2.曾參加過國內外競賽得獎...,待遇面議 ...,工研院服科中心_AI軟體研發工程師(U400)
991,台北市大同區台北 或 新竹,1. 特休全年十天起跳，最多三十天2. 事假、病假、喪假、婚假、產假、陪產假3. 年終獎金4...,樺鼎商業資訊股份有限公司,3年以上,1. SW Integration Development & Maintain2. IBP...,1. at least 3 years in project base SW develop...,待遇面議 ...,系統分析規劃顧問
992,新北市中和區中正路738號14樓,◆ 獎金/禮品類1.年終獎金2.三節獎金/禮品3.佳節禮品◆ 保險類1.勞保/健保2.員工團...,文曄集團_文曄科技股份有限公司,3年以上,【誰能成為WT菁英】1.國內外大學或研究所之資訊工程/資管等相關系所畢業。2.配合公司策略發...,1.熟資料庫語法2.具ETL設計經驗3.Oracle 資料庫開發經驗4.具 Tableau ...,待遇面議 ...,商業智慧(BI)分析師(IT)
993,台北市信義區信義路五段,本公司正職員工享有以下福利：● 年終獎金● 年度健檢● 三節禮金● 優雅舒適的辦公環境● 健...,17直播_英屬維京群島商麻吉一七股份有限公司臺灣分公司,2年以上,- QA experiences for mobile/web platforms with...,- 2 - 5 years of related experience.- Better f...,待遇面議 ...,QA Engineer 軟體測試工程師
994,台北市信義區莊敬路168號,我們提供優於法令規定及整體金融產業的福利措施如下:◎優於勞基法規定之休假制度◎提供多項激勵獎...,(總公司)南山人壽保險股份有限公司,3年以上,● 異質資料庫數據整合，建置資料倉儲與管理● 數據分析、應用之相關程式開發● 資料庫管理、設...,● 熟T-SQL語法及Stored Procedure開發● 熟MS SQL Server ...,待遇面議 ...,金融數據工程師
995,台北市信義區松仁路32號,尚未提供,Amazon Web Services Taiwan Limited_台灣亞馬遜網路服務有限公司,7年以上,Data Scientist - Machine LearningAWS Professio...,If you do not live in a market where we have a...,待遇面議 ...,Data Scientist - Machine Learning - AWS Profes...
